# Data Preparation For Bart


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root_dir = "/content/drive/MyDrive/Colab Notebooks/544_bart/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install rake-nltk
import pandas as pd
import numpy as np
from rake_nltk import Rake
import nltk

nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df = pd.read_csv(root_dir + 'ROCStories_winter2017 - ROCStories_winter2017.csv')
df

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight re...,He examined his habits to try and figure out t...,He realized he'd been eating too much fast foo...,He stopped going to burger places and started ...,"After a few weeks, he started to feel much bet..."
1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about ...
2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual e...,All of his clothes were either too formal or t...,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for...
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and ha...,Bill thought a truck would be better for what ...,Bobby pointed out two vehicles were much more ...,Bill was set in his ways with conventional thi...,He ended up buying the truck he wanted despite...
4,c71bb23b-7731-4233-8298-76ba6886cee1,Overcoming shortcomings,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in ...,He decided to learn to sing to overcome his ha...,He then made all his sermons into music and sa...,His congregation was delighted and so was he.
...,...,...,...,...,...,...,...
52660,134e8636-3617-43d8-ba6a-9a11b3b115b1,Flavor,The man liked the flavor.,He tried to recreate it at home.,He could not get the flavor right.,He asked the owner of the recipe for help.,The owner of the flavor sold him the recipe.
52661,4c317f76-ca42-4024-a4c2-12ec911cf89b,After Death,"After my friend's dad's funeral, I got in trou...",The principal said I wasn't allowed to leave s...,He found out I had my friend sign me out.,He told me I was getting detention.,I skipped detention all week.
52662,a18fd0d2-4d0c-4316-befe-e3d827fe699b,Janice breaks her wrist,Janice was out exercising for her big soccer g...,She was doing some drills with her legs.,While working out and exercising she slips on ...,She falls down and uses her wrist to break her...,She breaks her wrist in the process and goes t...
52663,2c14252b-4080-4fca-8765-537772018508,Jamie marries for love,Jamie is an american girl.,Jamie wants to get married to a mexican man.,Her family assumes it's because the man wants ...,Jamie insist that she is marrying him out of l...,Jamie gets married and they spent the rest of ...


In [ ]:
# df2 = pd.read_csv(root_dir + 'ROCStories__spring2016 - ROCStories_spring2016.csv')
# df2

In [ ]:
# df = df.append(df2, ignore_index=True)
# df

In [ ]:
# Uses stopwords for english from NLTK, and all punctuation characters by
# default
#max_length=1 so that we only get one word
r = Rake(max_length=1, min_length=1, include_repeated_phrases=False)

def rake_implement(s1, s2, s3, s4, s5, r):
    result = []
    result.append(get_word(r, s1))
    result.append(get_word(r, s2))
    result.append(get_word(r, s3))
    result.append(get_word(r, s4))
    result.append(get_word(r, s5))
    return ' '.join(result)

def get_word(r, s):
  r.extract_keywords_from_text(s)
  phrases = r.get_ranked_phrases()
  if(len(phrases) == 0):
    #if nothing is extracted, take first word that is not in stopwords (from the middle)
    sentence = s.split()
    for i in range(int(len(sentence)/2), len(sentence)):
      if not sentence[i] in stopwords.words('english'):
        return sentence[i].lower()
  else:
    return phrases[0]

#https://stackoverflow.com/questions/56836477/apply-nltk-rake-to-each-row-in-dataframe
df['storyline'] = df.apply(lambda x: rake_implement(x.sentence1, x.sentence2, x.sentence3, x.sentence4, x.sentence5, r), axis=1)

In [ ]:
df

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5,storyline
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight re...,He examined his habits to try and figure out t...,He realized he'd been eating too much fast foo...,He stopped going to burger places and started ...,"After a few weeks, he started to feel much bet...",put try realized started weeks
1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about ...,tom angry wall tom regret
2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual e...,All of his clothes were either too formal or t...,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for...,business formal pair perfectly marcus
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and ha...,Bill thought a truck would be better for what ...,Bobby pointed out two vehicles were much more ...,Bill was set in his ways with conventional thi...,He ended up buying the truck he wanted despite...,trailer needed much ways truck
4,c71bb23b-7731-4233-8298-76ba6886cee1,Overcoming shortcomings,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in ...,He decided to learn to sing to overcome his ha...,He then made all his sermons into music and sa...,His congregation was delighted and so was he.,pastor tried sing sundays delighted
...,...,...,...,...,...,...,...,...
52660,134e8636-3617-43d8-ba6a-9a11b3b115b1,Flavor,The man liked the flavor.,He tried to recreate it at home.,He could not get the flavor right.,He asked the owner of the recipe for help.,The owner of the flavor sold him the recipe.,flavor tried get recipe recipe
52661,4c317f76-ca42-4024-a4c2-12ec911cf89b,After Death,"After my friend's dad's funeral, I got in trou...",The principal said I wasn't allowed to leave s...,He found out I had my friend sign me out.,He told me I was getting detention.,I skipped detention all week.,trouble day found told week
52662,a18fd0d2-4d0c-4316-befe-e3d827fe699b,Janice breaks her wrist,Janice was out exercising for her big soccer g...,She was doing some drills with her legs.,While working out and exercising she slips on ...,She falls down and uses her wrist to break her...,She breaks her wrist in the process and goes t...,janice legs working wrist wrist
52663,2c14252b-4080-4fca-8765-537772018508,Jamie marries for love,Jamie is an american girl.,Jamie wants to get married to a mexican man.,Her family assumes it's because the man wants ...,Jamie insist that she is marrying him out of l...,Jamie gets married and they spent the rest of ...,jamie married man marrying spent


In [ ]:
def combineSen(row):
  return row['sentence1'] + row['sentence2'] + row['sentence3'] + row['sentence4'] + row['sentence5']
df['story'] = df.apply (lambda row: combineSen(row), axis=1)
df = df[['storytitle', 'storyline', 'story']]
df

,storytitle,storyline,story
0,David Drops the Weight,put try realized started weeks,David noticed he had put on a lot of weight re...
1,Frustration,tom angry wall tom regret,Tom had a very short temper.One day a guest ma...
2,Marcus Buys Khakis,business formal pair perfectly marcus,Marcus needed clothing for a business casual e...
3,Different Opinions,trailer needed much ways truck,Bobby thought Bill should buy a trailer and ha...
4,Overcoming shortcomings,pastor tried sing sundays delighted,John was a pastor with a very bad memory.He tr...
...,...,...,...
52660,Flavor,flavor tried get recipe recipe,The man liked the flavor.He tried to recreate ...
52661,After Death,trouble day found told week,"After my friend's dad's funeral, I got in trou..."
52662,Janice breaks her wrist,janice legs working wrist wrist,Janice was out exercising for her big soccer g...
52663,Jamie marries for love,jamie married man marrying spent,Jamie is an american girl.Jamie wants to get m...


In [ ]:
def concat_title_line(x):
    l = x['storytitle'].split()
    l.append('<EOT>')
    l.extend(x['storyline'].split())
    return ' '.join(l)

df['titleLineConcat'] = df.apply(lambda x: concat_title_line(x), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
df['titleLineConcat'][0]

'David Drops the Weight <EOT> put try realized started weeks'

In [ ]:
df

,storytitle,storyline,story,titleLineConcat
0,David Drops the Weight,put try realized started weeks,David noticed he had put on a lot of weight re...,David Drops the Weight <EOT> put try realized ...
1,Frustration,tom angry wall tom regret,Tom had a very short temper.One day a guest ma...,Frustration <EOT> tom angry wall tom regret
2,Marcus Buys Khakis,business formal pair perfectly marcus,Marcus needed clothing for a business casual e...,Marcus Buys Khakis <EOT> business formal pair ...
3,Different Opinions,trailer needed much ways truck,Bobby thought Bill should buy a trailer and ha...,Different Opinions <EOT> trailer needed much w...
4,Overcoming shortcomings,pastor tried sing sundays delighted,John was a pastor with a very bad memory.He tr...,Overcoming shortcomings <EOT> pastor tried sin...
...,...,...,...,...
52660,Flavor,flavor tried get recipe recipe,The man liked the flavor.He tried to recreate ...,Flavor <EOT> flavor tried get recipe recipe
52661,After Death,trouble day found told week,"After my friend's dad's funeral, I got in trou...",After Death <EOT> trouble day found told week
52662,Janice breaks her wrist,janice legs working wrist wrist,Janice was out exercising for her big soccer g...,Janice breaks her wrist <EOT> janice legs work...
52663,Jamie marries for love,jamie married man marrying spent,Jamie is an american girl.Jamie wants to get m...,Jamie marries for love <EOT> jamie married man...


In [ ]:
df.to_csv(root_dir+'train_title_line_story.csv', encoding='utf-8', index=False)

In [ ]:
#output notebook as pdf
!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf
colab_pdf('544_project_data_preparation.ipynb')

--2021-12-01 05:22:26--  https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1864 (1.8K) [text/plain]
Saving to: ‘colab_pdf.py’

colab_pdf.py        100%[===================>]   1.82K  --.-KB/s    in 0s      

2021-12-01 05:22:26 (28.6 MB/s) - ‘colab_pdf.py’ saved [1864/1864]

